## Scraping antara news

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
import re

In [2]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []
keywords='anies'


In [3]:
def scrape_links(page_number,keywords):
    url = f"https://www.antaranews.com/search?q={keywords}&page={page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article',{"class": "simple-post simple-big clearfix"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [4]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,keywords)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 15 links from page 1
Total Links: 15


In [5]:
def scrape_url(url,keywords):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1',{"class": "post-title"})
            if title_elem:
                title_text = title_elem.text
            else:
                title_text = "Title not found"
            # Author berita
            # author_elem = soup.find('div', {"class": "tags-wrapper"})
            # if author_elem:
            #     author_text = author_elem.find('span')
            #     author_text = author_text.text.split(':')[1]
            # else:
            author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('span', {"class": "article-date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            # category_elements = soup.find('ul', {"class": "breadcrumb"})
            # if category_elements:
            #     category_text = category_elements.find_all('li')
            #     category_text= category_text[2].get_text()
            # else:
            category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "post-content clearfix"})
            if body_elem:
                 content_text = body_elem.text
            else:
                content_text ="Content not found"
            nama_berita_match = re.search(r'https://www\.(\w+)\.com/', url)
            if nama_berita_match:
                nama_berita = nama_berita_match.group(1)
            else:
                nama_berita = "Nama_berita not found"
            
            results.append({'title': title_text,
                            'keywords': keywords,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                             'nama_berita' : nama_berita,
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [6]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,keywords))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [7]:
df = pd.DataFrame(results)
print("hasil data scrapping",len(results))
df.head(100)

hasil data scrapping 15


,title,keywords,author,category,date,content,nama_berita,link
0,"Tak ada hitam-putih dalam politik, yang ada pr...",anies,Author not found,Category not found,"Kamis, 28 September 2023 18:55 WIB",\n\t\t\t\tJakarta (ANTARA) - Batas akhir penda...,antaranews,https://www.antaranews.com/berita/3748029/tak-...
1,Pengamat: Ganjar unggul di bidang pangan dan l...,anies,Author not found,Category not found,"Kamis, 28 September 2023 15:08 WIB",\n\t\t\t\tJakarta (ANTARA) - Pengamat politik ...,antaranews,https://www.antaranews.com/berita/3747690/peng...
2,Said: Anies-Muhaimin di nikah putri Habib Rizi...,anies,Author not found,Category not found,"Kamis, 28 September 2023 17:22 WIB",\n\t\t\t\tJakarta (ANTARA) - Koordinator Badan...,antaranews,https://www.antaranews.com/berita/3747867/said...
3,Politikus PKB harap Said Aqil Siroj jadi Ketua...,anies,Author not found,Category not found,"Kamis, 28 September 2023 23:07 WIB",\n\t\t\t\tJakarta (ANTARA) - Ketua Bidang Huku...,antaranews,https://www.antaranews.com/berita/3748290/poli...
4,Pengamat: Demokrat tak beri dampak signifikan ...,anies,Author not found,Category not found,"Rabu, 27 September 2023 21:12 WIB",\n\t\t\t\tSurabaya (ANTARA) - Pengamat politik...,antaranews,https://www.antaranews.com/berita/3746859/peng...
5,SMRC: Ganjar-Mahfud unggul dengan 45 persen di...,anies,Author not found,Category not found,"Kamis, 28 September 2023 21:16 WIB",\n\t\t\t\tJakarta (ANTARA) - Saiful Mujani Res...,antaranews,https://www.antaranews.com/berita/3748209/smrc...
6,Ini untung rugi dua poros dalam pemilihan pres...,anies,Author not found,Category not found,"Rabu, 27 September 2023 22:56 WIB",\n\n \n\n\n\n\n\n\n\n\n\t\t\t\t\t\t\t\tANTARA ...,antaranews,https://www.antaranews.com/video/3746631/ini-u...
7,"Pasangan ""AMIN"" optimistis raup suara banyak d...",anies,Author not found,Category not found,"Kamis, 28 September 2023 15:25 WIB",\n\t\t\t\tBanyuwangi (ANTARA) - Pasangan bakal...,antaranews,https://www.antaranews.com/berita/3747708/pasa...
8,Safari politik Anies-Muhaimin di Ponpes,anies,Author not found,Category not found,"Kamis, 28 September 2023 14:00 WIB",Content not found,antaranews,https://www.antaranews.com/foto/3747615/safari...
9,Fahri Hamzah ungkap bakal banyak kejutan hingg...,anies,Author not found,Category not found,"Kamis, 28 September 2023 11:00 WIB",\n\t\t\t\tJakarta (ANTARA) - Wakil Ketua Umum ...,antaranews,https://www.antaranews.com/berita/3747495/fahr...


In [8]:
filtered_df = df[df['content'].str.contains(keywords, case=False)|
                 df['title'].str.contains(keywords, case=False)]
print('hasil filter',len(filtered_df))
filtered_df.head(10)

hasil filter 14


,title,keywords,author,category,date,content,nama_berita,link
0,"Tak ada hitam-putih dalam politik, yang ada pr...",anies,Author not found,Category not found,"Kamis, 28 September 2023 18:55 WIB",\n\t\t\t\tJakarta (ANTARA) - Batas akhir penda...,antaranews,https://www.antaranews.com/berita/3748029/tak-...
1,Pengamat: Ganjar unggul di bidang pangan dan l...,anies,Author not found,Category not found,"Kamis, 28 September 2023 15:08 WIB",\n\t\t\t\tJakarta (ANTARA) - Pengamat politik ...,antaranews,https://www.antaranews.com/berita/3747690/peng...
2,Said: Anies-Muhaimin di nikah putri Habib Rizi...,anies,Author not found,Category not found,"Kamis, 28 September 2023 17:22 WIB",\n\t\t\t\tJakarta (ANTARA) - Koordinator Badan...,antaranews,https://www.antaranews.com/berita/3747867/said...
3,Politikus PKB harap Said Aqil Siroj jadi Ketua...,anies,Author not found,Category not found,"Kamis, 28 September 2023 23:07 WIB",\n\t\t\t\tJakarta (ANTARA) - Ketua Bidang Huku...,antaranews,https://www.antaranews.com/berita/3748290/poli...
4,Pengamat: Demokrat tak beri dampak signifikan ...,anies,Author not found,Category not found,"Rabu, 27 September 2023 21:12 WIB",\n\t\t\t\tSurabaya (ANTARA) - Pengamat politik...,antaranews,https://www.antaranews.com/berita/3746859/peng...
5,SMRC: Ganjar-Mahfud unggul dengan 45 persen di...,anies,Author not found,Category not found,"Kamis, 28 September 2023 21:16 WIB",\n\t\t\t\tJakarta (ANTARA) - Saiful Mujani Res...,antaranews,https://www.antaranews.com/berita/3748209/smrc...
7,"Pasangan ""AMIN"" optimistis raup suara banyak d...",anies,Author not found,Category not found,"Kamis, 28 September 2023 15:25 WIB",\n\t\t\t\tBanyuwangi (ANTARA) - Pasangan bakal...,antaranews,https://www.antaranews.com/berita/3747708/pasa...
8,Safari politik Anies-Muhaimin di Ponpes,anies,Author not found,Category not found,"Kamis, 28 September 2023 14:00 WIB",Content not found,antaranews,https://www.antaranews.com/foto/3747615/safari...
9,Fahri Hamzah ungkap bakal banyak kejutan hingg...,anies,Author not found,Category not found,"Kamis, 28 September 2023 11:00 WIB",\n\t\t\t\tJakarta (ANTARA) - Wakil Ketua Umum ...,antaranews,https://www.antaranews.com/berita/3747495/fahr...
10,Muhaimin optimistis pasangan AMIN bisa menang ...,anies,Author not found,Category not found,"Kamis, 28 September 2023 22:57 WIB","\n\t\t\t\tJember, Jawa Timur (ANTARA) - Ketua ...",antaranews,https://www.antaranews.com/berita/3748281/muha...


Tempat Simpan Save

In [9]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'../tempat_simpan_nasional/antaranews_{keywords}_{current_datetime}.xlsx'
# df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')